In [4]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import torch
import cv2

# normalized keypoints

train = []
test = []
with open('../Data_Extraction/train.csv') as f:
    r = f.readline()
    while r!='':
        # print('r', r, 'r')
        number = int(r.split(' ')[0].split('_')[0][len('/home/swli2/slowfast/data/video'):])
        train.append(number)
        r = f.readline()
    
with open('../Data_Extraction/test.csv') as f:
    r = f.readline()
    while r!='':
        # print('r', r, 'r')
        number = int(r.split(' ')[0].split('_')[0][len('/home/swli2/slowfast/data/video'):])
        test.append(number)
        r = f.readline()


f = open('../Data_Extraction/test.csv')
buckets = 1

def bucket(x, count):
    if count==10:
        return x
    if count==5:
        return (x+1)//2
    if count==3:
        return min((x+2)//3, 3)
    if count==2:
        return (x+4)//5
    if count==1:
        return 0 if x==0 else 1
    
aggregate = True

root='../Landmarks/'
keypoints = list(pickle.load(open("../Landmarks/video0/1.pickle", 'rb')).keys())
if(aggregate):
    X_train = torch.zeros((0, 100, 212))
    X_test = torch.zeros((0, 100, 212))
else:
    X_train = torch.zeros((0, 212))
    X_test = torch.zeros((0, 212))
    
y_train = []
y_test = []
for dir in tqdm(os.listdir(root)):
    if os.path.isdir(os.path.join(root, dir)):
        file_path = os.path.join(root,dir)
        number = int(dir[len('video'):])
        train = True
        if number in test:
            train = False
        label = 0
        with open(f'{file_path}/multi_label.txt', 'rb') as f:
            label = int(f.readline())

        label = bucket(label, buckets)
        cur = None
        if(aggregate):
            cur = torch.zeros((1, 0, 212))
        correct_num = 0
        for file in os.listdir(file_path):
            if file.split('.')[1]=='png':
                correct_num += 1
        if file_path not in []:
            for i in range(1,correct_num+1):
                if not os.path.exists(f'{file_path}/{i}.pickle'):
                    # print(f"Error {file_path}")
                    continue
                absolute_landmark = pickle.load(open(f'{file_path}/{i}.pickle', 'rb'))
                features = []
                center = absolute_landmark['nose_tip']
                for feature in keypoints:
                    features.append((absolute_landmark[feature]['x'] - center['x'])/256) 
                    features.append((absolute_landmark[feature]['y'] - center['y'])/256)
                    # features.append(absolute_landmark[feature]['x'])
                    # features.append(absolute_landmark[feature]['y'])

                if(aggregate):
                    cur = torch.concat((cur, torch.Tensor(features).view(1, 1, 212)), axis=1)
                else:
                    if(train):
                        X_train = torch.concat((X_train, torch.Tensor(features).view(1, 212)), axis=0)
                        y_train.append(label)
                    else:
                        X_test = torch.concat((X_test, torch.Tensor(features).view(1, 212)), axis=0)
                        y_test.append(label)
        if(aggregate):
            # print(cur[0, :, :].shape)
            cur = cv2.resize(np.array(cur[0, :, :]), (212, 100), interpolation = cv2.INTER_LINEAR)
            
            # print(cur.shape)
            if(train):
                X_train = torch.concat((X_train, torch.unsqueeze(torch.Tensor(cur), dim=0)), axis=0)
                y_train.append(label)
            else:
                X_test = torch.concat((X_test, torch.unsqueeze(torch.Tensor(cur), dim=0)), axis=0)
                y_test.append(label)

y_mc_train = torch.Tensor(y_train)
y_mc_test = torch.Tensor(y_test)

X_train_orig = torch.clone(X_train)
X_test_orig = torch.clone(X_test)

print(X_train.shape, y_mc_train.shape, X_test.shape, y_mc_test.shape)



100%|██████████| 246/246 [01:45<00:00,  2.34it/s]

torch.Size([72, 100, 212]) torch.Size([72]) torch.Size([174, 100, 212]) torch.Size([174])


In [ ]:
X_train = X_train_orig
X_test = X_test_orig

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np


def normalize(X):
    return (X - torch.mean(X)) / torch.std(X)

def transform(X):
    ret = torch.zeros_like(X)
    ret[:, :, 0:33*2:2] = normalize(X[:, :, 0:33*2:2])
    ret[:, :, 1:33*2:2] = normalize(X[:, :, 1:33*2:2])
    ret[:, :, 33*2:51*2:2] = normalize(X[:, :, 33*2:51*2:2])
    ret[:, :, 33*2+1:51*2:2] = normalize(X[:, :, 33*2+1:51*2:2])
    ret[:, :, 51*2:72*2:2] = normalize(X[:, :, 51*2:72*2:2])
    ret[:, :, 51*2+1:72*2:2] = normalize(X[:, :, 51*2+1:72*2:2])
    ret[:, :, 72*2:87*2:2] = normalize(X[:, :, 72*2:87*2:2])
    ret[:, :, 72*2+1:87*2:2] = normalize(X[:, :, 72*2+1:87*2:2])
    ret[:, :, 87*2:106*2:2] = normalize(X[:, :, 87*2:106*2:2])
    ret[:, :, 87*2+1:106*2:2] = normalize(X[:, :, 87*2+1:106*2:2])
    return ret


def calculate_jitter_ppqf(X_test, frames=5):
    jitter_ppqf = torch.zeros(X_test.shape)
    for i in range(X_test.shape[0]):
        for j in range(X_test.shape[1]):
            if j==0:
                jitter_ppqf[i, j, :] = X_test[i, j, :]
            else:
                if j<frames:
                    jitter_ppqf[i, j, :] = X_test[i, j, :] - torch.mean(X_test[i, 0:j+1, :], dim=0)
                else:
                    jitter_ppqf[i, j, :] = X_test[i, j, :] - torch.mean(X_test[i, j-frames:j+1, :], dim=0)
    return jitter_ppqf

def get_motion_clusters(X, n_components=2, frames = 10):
    ''' Calculate the difference for each keypoint between frames and return the mean and stdev for each facial region.'''
    ret = torch.zeros((X.shape[0], X.shape[1], 5*n_components))

    diff_jaw = (calculate_jitter_ppqf(X[:, :, 0:33], frames))
    diff_eye = (calculate_jitter_ppqf(X[:, :, 33:51], frames))
    diff_mouth = (calculate_jitter_ppqf(X[:, :, 51:72], frames))
    diff_nose = (calculate_jitter_ppqf(X[:, :, 72:87], frames))
    diff_eye2 = (calculate_jitter_ppqf(X[:, :, 87:106], frames))

    ret[:, :, 0] = torch.mean(diff_jaw, dim=2)
    ret[:, :, 1] = torch.mean(diff_eye, dim=2)
    ret[:, :, 2] = torch.mean(diff_mouth, dim=2)
    ret[:, :, 3] = torch.mean(diff_nose, dim=2)
    ret[:, :, 4] = torch.mean(diff_eye2, dim=2)
    ret[:, :, 5] = torch.std(diff_jaw, dim=2)
    ret[:, :, 6] = torch.std(diff_eye, dim=2)
    ret[:, :, 7] = torch.std(diff_mouth, dim=2)
    ret[:, :, 8] = torch.std(diff_nose, dim=2)
    ret[:, :, 9] = torch.std(diff_eye2, dim=2)

    return ret

def remove_jaw(X):
    return X[:, :, 33*2:]


X_cluster_train = get_motion_clusters(X_train)
X_cluster_test = get_motion_clusters(X_test)

X_train = normalize(X_train)
X_test = normalize(X_test)

X_train = remove_jaw(X_train)
X_test = remove_jaw(X_test)


# X_train = torch.cat((X_train, X_cluster_train), dim=2)
# X_test = torch.cat((X_test, X_cluster_test), dim=2)

print(X_train.shape, y_mc_train.shape, X_test.shape, y_mc_test.shape)

torch.Size([72, 100, 146]) torch.Size([72]) torch.Size([174, 100, 146]) torch.Size([174])


In [ ]:
import torch.nn as nn
import torch
from torchmetrics.classification import BinaryAUROC, AveragePrecision
# CUDA_LAUNCH_BLOCKING=1

class concatlin(nn.Module):
    def __init__(self, dim=512, frames=8):
        super().__init__()
        self.frames = frames
        self.lin1 = nn.Linear(X_train.shape[2], dim)
        self.lin2 = nn.Linear(frames*dim, buckets+1)
        self.ReLU = nn.ReLU()
        self.bn = nn.BatchNorm1d(frames*dim)


    def forward(self, x):
        x = x[:, ::x.shape[1]//self.frames, :]
        x = x[:, :8, :]
        res = self.lin1(x)
        res = torch.flatten(res, start_dim=1)
        res = self.ReLU(res)
        res = self.bn(res)
        res = self.lin2(res)
        return res

class lin(nn.Module):
    def __init__(self):
        super().__init__()
        self.ReLU = nn.ReLU()
        self.lin1 = nn.Linear(X_train.shape[2], 30)
        self.lin2 = nn.Linear(60, 30)
        self.lin3 = nn.Linear(30, buckets+1)
        self.bn = nn.BatchNorm1d(buckets+1)
        

    def forward(self, x):
        res = self.ReLU(self.lin1(x))
        # res = self.ReLU(self.lin2(res))
        res = self.lin3(res)
        # print(res.shape)
        if(aggregate):
            res = torch.mean(res, axis=1)
        # print(res.shape)
        res = self.bn(res)
        return res

t1accs = []
mses = []
f1s = []
aurocs = []
mmses = []
baurocs = []
bauprcs = []
bf1s = []
baccs = []
brecalls = []
num_iters = 200
dim = 512
frames = 8
for _ in range(7):

    mt = 'lin'
    model = concatlin(dim=dim, frames=frames)

    model.train()
    model.to('cuda')
    optim = torch.optim.Adam(model.parameters(), lr=3e-4)

    use_mse = False

    input = torch.Tensor(X_train).to('cuda')
    if(mt == 'CNN'):
        input = input.swapaxes(1, 2)

    target = torch.Tensor(y_mc_train).type(torch.LongTensor).to('cuda')

    # print(input.shape, target.shape)

    weights = torch.Tensor([1, (36/8), (36/5), (36/6), (36/12), (36/5)]).to('cuda')
    weights = torch.Tensor([1, 1]).to('cuda')

    def loss_func(pred, target, l=0.5):
        # print(pred.shape, target.shape)
        # print(target)
        targets = torch.nn.functional.one_hot(target, pred.shape[1])
        targets = torch.mul(targets, weights)
        spred = torch.clip(nn.Softmax(dim=1)(pred), 1e-20, 1-1e-20)
        # print(spred.shape, pred.shape, t.shape)
        wce = -torch.mean(targets*torch.log(spred))

        # mce = nn.CrossEntropyLoss()(pred, target)

        t = (target!=0).type(torch.float32)
        bce = nn.BCELoss()(1-spred[:, 0], t)

        mpred = torch.sum((torch.softmax(pred, axis=1) * torch.Tensor(np.arange(buckets+1)).to('cuda')), axis=1)
        mse = torch.mean((mpred - target)**2)
        # m_bce = nn.BCEWithLogitsLoss()(torch.clip(1 - mpred, min = 1e-20), (target==0).type(torch.float32))
        if(use_mse):
            return mse + l*bce
        else:
            return wce + l*bce

    train_err = []
    for i in range(num_iters):
        model.to('cuda')
        model.train()
        if(not aggregate):
            ind = np.random.choice(6697, 1024, replace=False)
            pred = model(input[ind, :])
        else:
            ind = np.random.choice(72, 64, replace=False)
            # ind = np.random.randint(0, 72, 32)
            # ind = np.arange(0, 72)
            pred = model(input[ind, :, :])
        # if(dtu == 'mae_full'):
        #     pred = model(input[ind, :, :])
        # else:
        #     pred = model(input[ind, :])
        optim.zero_grad()
        loss = loss_func(pred, target[ind], l=0)
        loss.backward()
        optim.step()
        if(i%num_iters==num_iters-1):
            print(loss.item())
            with torch.no_grad():
                model.eval()
                model.to('cuda')
                inp = torch.Tensor(X_test).to('cuda')
                if(mt == 'CNN'):
                    inp = inp.swapaxes(1, 2)
                y = torch.Tensor(y_mc_test).to('cuda')

                pred = model(inp)#.reshape(y.shape)
                s = torch.softmax(pred, axis=1)
                if(not use_mse):
                    pred = torch.argmax(pred, axis=1)
                else:
                    pred = torch.softmax(pred, axis=1)
                    pred = torch.round(torch.sum((pred * torch.Tensor(np.arange(buckets+1)).to('cuda')), axis=1))
                tmse = nn.MSELoss()(pred, y)
                # print("Test MSE:", tmse)
                # pred = torch.round(pred)
                # print(pred)
                # print(y)

                acc = torch.mean((pred==y).type(torch.float32))
                # print("Test ACC:", acc)
                # print()
                t1macro = 0
                f1macro = 0
                aurocmacro = 0
                msemacro = 0
                for b in range(buckets+1):
                    precision = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((pred==b).type(torch.float32))
                    recall = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((y==b).type(torch.float32))
                    f1 = (2*precision*recall)/(precision+recall)
                    f1 = torch.nan_to_num(f1, 0, 0, 0)
                    auroc = BinaryAUROC()(s[:, b] , (y==b).type(torch.float32))
                    mse = torch.sum((y==b)*(pred - y)**2)/torch.sum((y==b).type(torch.float32))
                    # print("Class: " + str(b), "Precision: " + str(precision.item()), "Recall: " + str(recall.item()), "F1: " + str(f1.item()), "AUROC: " + str(auroc.item()), "MSE: " + str(mse.item()))
                    t1macro+=recall
                    f1macro+= f1
                    aurocmacro+=auroc
                    msemacro+=mse
                # print("Macros (t1, f1, auroc, mse): \nT1: " + str(t1macro.item()/(buckets+1))+ ", F1: " 
                #         + str(f1macro.item()/(buckets+1))+ ", AUROC: " + str(aurocmacro.item()/(buckets+1))+ ", MSE: " 
                #         + str(msemacro.item()/(buckets+1)) + "\n")

                t = (y!=0).type(torch.float32)


                # print(y.shape)
                
                # print("Metrics of Interest (MSE, f1, auroc, mse, Bin AUROC): \nTest MSE: " + str(tmse.item()) + " F1: " 
            # + str(f1macro.item()/(buckets+1))+ ", AUROC: " + str(aurocmacro.item()/(buckets+1))+ ", MSE: " 
            # + str(msemacro.item()/(buckets+1)) + ", Bin AUROC: " + str(BinaryAUROC()(1-s[:, 0], t).item()) + "\n")
                
                # print("Binary acc", (torch.sum(torch.logical_and(pred==0, y==0).type(torch.float32))+torch.sum(torch.logical_and(pred!=0, y!=0).type(torch.float32)))/y.shape[0])
                # print("Binary AUROC",  BinaryAUROC()(1-s[:, 0], t))

                t1accs.append(acc.item())
                mses.append(tmse.item())
                f1s.append(f1macro.item()/(buckets+1))
                aurocs.append(aurocmacro.item()/(buckets+1))
                mmses.append(msemacro.item()/(buckets+1))
                baurocs.append(BinaryAUROC()(1-s[:, 0], t).item())
                bauprcs.append(AveragePrecision(task="binary")(1-s[:, 0], t.long()).item())
                
                bin_prec = torch.sum(torch.logical_and(pred!=0, y!=0).type(torch.float32))/torch.sum((pred!=0).type(torch.float32))
                bin_rec = torch.sum(torch.logical_and(pred!=0, y!=0).type(torch.float32))/torch.sum((y!=0).type(torch.float32))
                bin_f1 = (2*bin_prec*bin_rec)/(bin_prec+bin_rec)
                bf1s.append(bin_f1.item())
                bin_acc = torch.sum(torch.logical_or(torch.logical_and(pred==0, y==0).type(torch.float32), torch.logical_and(pred!=0, y!=0).type(torch.float32)))/len(y)
                baccs.append(bin_acc.item())
                brecalls.append(bin_rec.item())
                
                inp = torch.Tensor(X_train).to('cuda')
                if(mt == 'CNN'):
                    inp = inp.swapaxes(1, 2)
                # y = torch.Tensor(y_train)
                y = torch.Tensor(y_mc_train).to('cuda')

                pred = model(inp)#.reshape(y.shape)
                if(not use_mse):
                    pred = torch.argmax(pred, axis=1)
                else:
                    pred = torch.softmax(pred, axis=1)
                    pred = torch.round(torch.sum((pred * torch.Tensor(np.arange(buckets+1)).to('cuda')), axis=1))
                # print("\nTrain MSE:", nn.MSELoss()(pred, y))
                # pred = torch.round(pred)
                # print(pred)
                # print(y)

                acc = torch.mean((pred==y).type(torch.float32))
                train_err.append(acc.item())
                # print("Train ACC:", acc)
                macro = 0
                for b in range(buckets+1):
                    precision = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((pred==b).type(torch.float32))
                    recall = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((y==b).type(torch.float32))
                    f1 = (2*precision*recall)/(precision+recall)
                    # print("Class: " + str(b), "Precision: " + str(precision.item()), "Recall: " + str(recall.item()), "F1: " + str(f1.item()))
                    macro+=recall
                print("Train Macro: " + str(macro/(buckets+1)))
                print("\n\n")

print('t1accs', np.mean(t1accs), 2*np.std(t1accs))
print('f1s', np.mean(f1s), 2*np.std(f1s))
print('aurocs', np.mean(aurocs), 2*np.std(aurocs))
print('mses', np.mean(mses), 2*np.std(mses))
# print(np.mean(mmses), 2*np.std(mmses))
print()
print('baccs', np.mean(baccs), 2*np.std(baccs))
print('bf1s', np.mean(bf1s), 2*np.std(bf1s))
print('baurocs', np.mean(baurocs), 2*np.std(baurocs))
print('bauprcs', np.mean(bauprcs), 2*np.std(bauprcs))
print('brecalls', np.mean(brecalls), 2*np.std(brecalls))


8.559026173315942e-05
Train Macro: tensor(1., device='cuda:0')



6.24652166152373e-05
Train Macro: tensor(1., device='cuda:0')



0.0001220284029841423
Train Macro: tensor(1., device='cuda:0')



7.229037873912603e-05
Train Macro: tensor(1., device='cuda:0')



5.282011261442676e-05
Train Macro: tensor(1., device='cuda:0')



5.708743628929369e-05
Train Macro: tensor(1., device='cuda:0')



7.729962817393243e-05
Train Macro: tensor(1., device='cuda:0')



t1accs 0.6075533628463745 0.04626929249621942
f1s 0.5165990335600716 0.030323675327138998
aurocs 0.7078942826816014 0.020260812051910528
mses 0.39244662863867624 0.04626928524379307

baccs 0.6075533628463745 0.04626929249621942
bf1s 0.30738223450524466 0.029079680531369383
baurocs 0.7078831791877747 0.020295826316782936
bauprcs 0.2056723407336644 0.018043187236332983
brecalls 0.7210884519985744 0.09426126282071315


: 

In [ ]:
# t1accs 0.4146141154425485 0.02077030461913147
# f1s 0.15125993745667593 0.005472780816685548
# aurocs 0.6122576963333858 0.006654856270181568
# mses 7.112479482378278 0.5202743544326753

# baccs 0.47290640217917307 0.015920123919114983
# bf1s 0.23739901185035706 0.013773630591814195
# baurocs 0.5913921168872288 0.015979205375873568
# bauprcs 0.15737075890813554 0.006213324796722303
# brecalls 0.6802721279008048 0.06665277814373366